This script takes regression data for u,v variables (x coordinate and y coordinate vectors) such as u-wind and v-wind or northward moisture transport and eastward moisture transport and creates a combined magnitude/direciton vector to help explain regressions better.

In [1]:
# import functions
# OS interaction and time
import os
import sys
import cftime
import datetime
import time
import glob
import dask
import dask.bag as db
import calendar
import importlib

# math and data
import math
import numpy as np
import netCDF4 as nc
import xarray as xr
import scipy as sp
import scipy.linalg
from scipy.signal import detrend
import pandas as pd
import pickle as pickle
from sklearn import linear_model
import matplotlib.patches as mpatches
from shapely.geometry.polygon import LinearRing
import statsmodels.stats.multitest as multitest

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg
from matplotlib.colors import TwoSlopeNorm
import matplotlib.cm as cm

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.axes_divider import HBoxDivider
import mpl_toolkits.axes_grid1.axes_size as Size
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

# random
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

my_era5_path = '/glade/u/home/zcleveland/scratch/ERA5/'  # path to subset data
misc_data_path = '/glade/u/home/zcleveland/scratch/misc_data/'  # path to misc data
plot_out_path = '/glade/u/home/zcleveland/NAM_soil-moisture/ERA5_analysis/plots/'  # path to generated plots
scripts_main_path = '/glade/u/home/zcleveland/NAM_soil-moisture/scripts_main/'  # path to my dicts, lists, and functions

# import variable lists and dictionaries
if scripts_main_path not in sys.path:
    sys.path.insert(0, scripts_main_path)  # path to file containing these lists/dicts
if 'get_var_data' in sys.modules:
    importlib.reload(sys.modules['get_var_data'])
if 'my_functions' in sys.modules:
    importlib.reload(sys.modules['my_functions'])
if 'my_dictionaries' in sys.modules:
    importlib.reload(sys.modules['my_dictionaries'])
if 'order_years' in sys.modules:
    importlib.reload(sys.modules['order_years'])

# import common functions that I've created
from get_var_data import get_var_data, get_var_files, open_var_data, subset_var_data, time_to_year_month_avg, time_to_year_month_sum, time_to_year_month
from my_functions import month_num_to_name, ensure_var_list
from order_years import *  # order_years(var, months, **kwargs)

# import lists and dictionaries
import my_dictionaries
# my lists
sfc_instan_list = my_dictionaries.sfc_instan_list  # instantaneous surface variables
sfc_accumu_list = my_dictionaries.sfc_accumu_list  # accumulated surface variables
pl_var_list = my_dictionaries.pl_var_list  # pressure level variables
invar_var_list = my_dictionaries.invar_var_list  # invariant variables
NAM_var_list = my_dictionaries.NAM_var_list  # NAM-based variables
region_avg_list = my_dictionaries.region_avg_list  # region IDs for regional averages
flux_var_list = my_dictionaries.flux_var_list  # flux variables that need to be flipped (e.g., sensible heat so that it's positive up instead of down
misc_var_list = my_dictionaries.misc_var_list  # misc variables
# my dictionaries
var_dict = my_dictionaries.var_dict  # variables and their names
var_units = my_dictionaries.var_units  # variable units
region_avg_dict = my_dictionaries.region_avg_dict  # region IDs and names
region_avg_coords = my_dictionaries.region_avg_coords  # coordinates for regions
region_colors_dict = my_dictionaries.region_colors_dict  # colors to plot for each region

In [11]:
def main(var1, var1_region, var1_months, var2, var2_region, var2_months, **kwargs):
    # get kwargs
    save_nc = kwargs.get('save_nc', False)
    save_png = kwargs.get('save_png', False)
    show_png = kwargs.get('show_png', False)
    overwrite_flag = kwargs.get('overwrite_flag', False)

    # check whether var1 or var2 are the list to combine.
    vars_list = check_vars_list(var1, var2)
    if not vars_list:
        print(f'Exactly one of var1 or var2 must be a list, but not none or both:
                \nvar1: {var1}\t--\tvar2: {var2}')

    # get month names for vars
    var_month_names = month_num_to_name(var, var_months)
    uvar_month_names = month_num_to_name(uvar, uvar_months)
    vvar_month_names = month_num_to_name(vvar, vvar_months)

    var_uvar_fn = get_out_fn(var, var_month_names, var_region,
                             uvar, uvar_month_names, uvar_region,
                             var1_level = kwargs.get('var_level', ''), var2_level = kwargs.get('uvar_level', ''), **kwargs)
    var_vvar_fn = get_out_fn(var, var_month_names, var_region,
                             vvar, vvar_month_names, vvar_region,
                             var1_level = kwargs.get('var_level', ''), var2_level = kwargs.get('vvar_level', ''), **kwargs)

    var_uvar_fp = get_out_fp(var_uvar_fn, var_region, uvar_region)
    var_vvar_fp = get_out_fp(var_vvar_fn, var_region, vvar_region)

    var_uvar_data = xr.open_dataset(var_uvar_fp)
    var_vvar_data = xr.open_dataset(var_vvar_fp)
    return var_uvar_data, var_vvar_data

In [ ]:
def combine

In [3]:
def check_vars_list(var1, var2, **kwargs):
    if isinstance(var1, list) and isinstance(var2, list):
        if not kwargs.get('compat_flag', False):
            raise ValueError("Both var1 and var2 cannot be lists")
    if isinstance(var1, list):
        return var1
    elif isinstance(var2, list):
        return var2
    else:
        return None  # if neither are a list, return None

In [6]:
# define a function to create the output filenames
def get_out_fn(var1, var1_months, var1_region, var2, var2_months, var2_region, **kwargs):
    # get optional arguments for naming
    var1_level = kwargs.get('var1_level', '')
    var2_level = kwargs.get('var2_level', '')

    # temporarily get detrend type while I decide which to use
    detrend_type = kwargs.get('detrend_type', '')

    # create core of output file name
    fn_list = [str(var1), str(var1_level), str(var1_months), str(var1_region),
               str(var2), str(var2_level), str(var2_months), str(var2_region),
               str(detrend_type)]
    fn_core = '_'.join([i for i in fn_list if i != ''])

    out_fn = f'regress_{fn_core}.nc'

    return out_fn

In [7]:
# define a function to create the output filepaths
def get_out_fp(out_fn, var1_region, var2_region, **kwargs):

    # check region for where to save .nc and .png files
    if (var1_region == 'global') or (var2_region == 'global'):
        out_fp = os.path.join(my_era5_path, 'regressions/global', out_fn)
    elif (var1_region == 'dsw') or (var2_region == 'dsw'):
        out_fp = os.path.join(my_era5_path, 'regressions/dsw', out_fn)
    elif (var1_region in region_avg_list) and (var2_region in region_avg_list):
        out_fp = os.path.join(my_era5_path, 'regressions/regions', out_fn)
    else:
        out_fp = None

    return out_fp

In [ ]:
u = time_to_year_month_avg(get_var_data('viwve', 'dsw')).sel(month=[5, 6]).mean('month', skipna=True)
v = time_to_year_month_avg(get_var_data('viwvn', 'dsw')).sel(month=[5, 6]).mean('month', skipna=True)
onset = get_var_data('onset', 'cp')
u_regress = xr.open_dataset('/glade/u/home/zcleveland/scratch/ERA5/regressions/dsw/regress_viwve_MJ_dsw_onset_cp_zscore.nc')
v_regress = xr.open_dataset('/glade/u/home/zcleveland/scratch/ERA5/regressions/dsw/regress_viwvn_MJ_dsw_onset_cp_zscore.nc')